In [ ]:
import sys
from pathlib import Path
import pandas as pd

def find_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(10):
        if (cur / "DATA" / "processed" / "master_df.parquet").exists():
            return cur
        if (cur / ".git").exists():
            return cur
        cur = cur.parent
    raise RuntimeError("Could not find repo root")

project_root = find_repo_root(Path.cwd())
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.diagnostics.seasonality import analyze_seasonality, plot_seasonality_heatmap

df = pd.read_parquet(project_root / "DATA" / "processed" / "master_df.parquet").sort_index()
df_diff = df.diff().dropna()
print("Data loaded + differenced")


 Data Loaded and Differenced


In [3]:
# Define key assets to test
targets = {
    "USD Funding (Repo)": [c for c in df.columns if "repo" in c.lower()][0],
    "Bond Yields (US 10Y)": [c for c in df.columns if "US10Y" in c or "bond_yields" in c][0],
    "Market Stress (MOVE)": [c for c in df.columns if "move" in c.lower()][0]
}

results_list = []

for category, col in targets.items():
    res = analyze_seasonality(df_diff[col], name=category)
    results_list.append(res)
    
    print(f"--- {category} Analysis ---")
    print(f"Year-End Volatility Ratio (Dec/Jan vs Rest): {res['ye_ratio']:.2f}x")
    print(f"Statistical Significance (p-value): {res['p_value']:.4f}")
    print("Is Seasonality Material?", "🚨 YES" if res['significant'] else "✅ NO")
    print("")

# Save these numbers for your markdown summary!

--- USD Funding (Repo) Analysis ---
Year-End Volatility Ratio (Dec/Jan vs Rest): nanx
Statistical Significance (p-value): nan
Is Seasonality Material? ✅ NO

--- Bond Yields (US 10Y) Analysis ---
Year-End Volatility Ratio (Dec/Jan vs Rest): nanx
Statistical Significance (p-value): nan
Is Seasonality Material? ✅ NO

--- Market Stress (MOVE) Analysis ---
Year-End Volatility Ratio (Dec/Jan vs Rest): nanx
Statistical Significance (p-value): nan
Is Seasonality Material? ✅ NO



c:\Users\avani\Cross-Country-Rates-Relative-Value-NUSInvest2526\src\diagnostics\seasonality.py:27: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  stat, p_value = stats.levene(ye_data, rest_data)
c:\Users\avani\Cross-Country-Rates-Relative-Value-NUSInvest2526\src\diagnostics\seasonality.py:27: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  stat, p_value = stats.levene(ye_data, rest_data)
c:\Users\avani\Cross-Country-Rates-Relative-Value-NUSInvest2526\src\diagnostics\seasonality.py:27: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  stat, p_value = stats.levene(ye_data, rest_data)


In [5]:
# 1. Check Repo for Year-End Funding Stress
# Look for bright red spots in Month 12 (December)
plot_seasonality_heatmap(df_diff, "repo", "Funding Stress (Repo)")

# 2. Check Bond Yields for General Seasonality
plot_seasonality_heatmap(df_diff, "bond_yields", "Bond Yields")

ValueError: zero-size array to reduction operation fmin which has no identity

<Figure size 1000x600 with 0 Axes>